#  Forward pass

__Автор задач: Блохин Н.В. (NVBlokhin@fa.ru)__

Материалы:
* Deep Learning with PyTorch (2020) Авторы: Eli Stevens, Luca Antiga, Thomas Viehmann
* https://pytorch.org/docs/stable/generated/torch.matmul.html
* https://machinelearningmastery.com/choose-an-activation-function-for-deep-learning/
* https://machinelearningmastery.com/loss-and-loss-functions-for-training-deep-learning-neural-networks/

## Задачи для совместного разбора

In [1]:
import torch as th

In [2]:
t = th.randint(0, 10, size=(2, 2))
t

tensor([[8, 2],
        [4, 8]])

In [3]:
t.sum(axis=1)

tensor([10, 12])

1\. Используя операции над матрицами и векторами из библиотеки `torch`, реализуйте нейрон с заданными весами `weights` и `bias`. Пропустите вектор `inputs` через нейрон и выведите результат.

In [ ]:
import torch as th

In [ ]:
class Neuron:
  def __init__(self, weights, bias):
    self.weights = weights
    self.bias = bias

  def forward(self, inputs): # <реализовать логику нейрона>
    return inputs @ self.weights + self.bias

In [ ]:
inputs = th.tensor([1.0, 2.0, 3.0, 4.0])
weights = th.tensor([-0.2, 0.3, -0.5, 0.7])
bias = 3.14

In [ ]:
neuron = Neuron(weights, bias)
neuron.forward(inputs)

tensor(4.8400)

2\. Используя операции над матрицами и векторами из библиотеки `torch`, реализуйте функцию активации ReLU:

![](https://wikimedia.org/api/rest_v1/media/math/render/svg/f4353f4e3e484130504049599d2e7b040793e1eb)

Создайте матрицу размера (4,3), заполненную числами из стандартного нормального распределения, и проверьте работоспособность функции активации.

In [ ]:
class ReLU:
  def forward(self, inputs):
    return th.clip(inputs, min=0)

In [ ]:
t = th.normal(0, 1, size=(4, 3))
t

tensor([[ 0.4023,  0.0824, -2.0781],
        [-0.0438, -0.3460, -0.9756],
        [ 0.1082, -0.3124, -2.0756],
        [ 2.0965, -0.8533, -0.4545]])

In [ ]:
ReLU().forward(t)

tensor([[0.4023, 0.0824, 0.0000],
        [0.0000, 0.0000, 0.0000],
        [0.1082, 0.0000, 0.0000],
        [2.0965, 0.0000, 0.0000]])

3\. Используя операции над матрицами и векторами из библиотеки `torch`, реализуйте функцию потерь MSE:

![](https://wikimedia.org/api/rest_v1/media/math/render/svg/e258221518869aa1c6561bb75b99476c4734108e)
где $Y_i$ - правильный ответ для примера $i$, $\hat{Y_i}$ - предсказание модели для примера $i$, $n$ - количество примеров в батче.

In [ ]:
class MSELoss:
  def forward(self, y_pred, y_true):
    return ((y_pred - y_true)**2).mean()

In [ ]:
y_pred = th.tensor([1.0, 3.0, 5.0])
y_true = th.tensor([2.0, 3.0, 4.0])

In [ ]:
MSELoss().forward(y_pred, y_true)

tensor(0.6667)

## Задачи для самостоятельного решения

In [ ]:
import torch as th

### Cоздание полносвязных слоев

<p class="task" id="1_1"></p>

1\.1 Используя операции над матрицами и векторами из библиотеки `torch`, реализуйте полносвязный слой из `n_neurons` нейронов с `n_features` весами у каждого нейрона (инициализируются из стандартного нормального распределения) и опциональным вектором смещения.

$$y = xW^T + b$$

Пропустите вектор `inputs` через слой и выведите результат. Результатом прогона сквозь слой должна быть матрица размера `batch_size` x `n_neurons`.

In [ ]:
class Linear:
    def __init__(self, n_neurons, n_features, bias: bool = False):
        self.n_neurons = n_neurons
        self.n_features = n_features
        self.weights = th.normal(0, 1, size=(n_neurons, n_features))
        self.bias_bool = bias
        if bias:
          self.bias = th.normal(0, 1, size=(self.n_neurons,))

    def forward(self, inputs):
        if self.bias_bool:
          return inputs @ (self.weights.T) + self.bias
        else:
          return inputs @ (self.weights).T

In [ ]:
# inputs: batch_size(3) x 4
inputs = th.tensor([[1, 2, 3, 2.5],
                       [2, 5, -1, 2],
                       [-1.5, 2.7, 3.3, -0.8]])
Lin = Linear(inputs.shape[0], inputs.shape[1], bias=True)

In [ ]:
Lin.forward(inputs)

tensor([[ 1.3077, -0.1480,  3.4065],
        [ 6.9648, 10.4536,  6.4605],
        [ 8.4424,  0.9023, -1.5492]])

<p class="task" id="1_2"></p>

1\.2 Используя решение предыдущей задачи, создайте 2 полносвязных слоя и пропустите тензор `inputs` последовательно через эти два слоя. Количество нейронов в первом слое выберите произвольно, количество нейронов во втором слое выберите так, чтобы результатом прогона являлась матрица `batch_size x 7`.

In [ ]:
l1 = Linear(inputs.shape[0], inputs.shape[1], bias=True)
l2 = Linear(n_neurons=7, n_features=inputs.shape[0], bias=True)

In [ ]:
l2.forward(l1.forward(inputs))

tensor([[ -8.5785,   3.6290, -21.0871,   4.1429,   8.5917,  -0.7564,   6.9599],
        [ 19.0256,   0.2378, -23.0537,  -0.4755,   1.2056,  12.8846, -12.0435],
        [-39.3855,   5.1582,  -6.5101,   2.9194,  17.1259, -22.4057,  28.6506]])

### Создание функций активации

<p class="task" id="2_1"></p>

2\.1 Используя операции над матрицами и векторами из библиотеки `torch`, реализуйте функцию активации softmax:

![](https://wikimedia.org/api/rest_v1/media/math/render/svg/6d7500d980c313da83e4117da701bf7c8f1982f5)

$$\overrightarrow{x} = (x_1, ..., x_J)$$

Создайте матрицу размера (4,3), заполненную числами из стандартного нормального распределения, и проверьте работоспособность функции активации. Строки матрицы трактовать как выходы линейного слоя некоторого классификатора для 4 различных примеров. Функция должна применяться переданной на вход матрице построчно.

In [ ]:
t = th.normal(0, 1, size=(4, 3))
t

tensor([[ 0.3269,  0.9085, -0.3805],
        [ 0.5275,  1.2475,  0.2329],
        [-0.0398, -0.9823,  0.3925],
        [ 1.9472, -0.0671, -0.4907]])

In [ ]:
class Softmax:
    def forward(self, inputs):
      sums = inputs.sum(axis=1)
      return th.exp(inputs) / th.exp(inputs).sum(axis=1, keepdim=True)

In [ ]:
Softmax().forward(t)

tensor([[0.3047, 0.5451, 0.1502],
        [0.2632, 0.5407, 0.1960],
        [0.3412, 0.1330, 0.5258],
        [0.8192, 0.1093, 0.0716]])

<p class="task" id="2_1"></p>

2.2 Используя операции над матрицами и векторами из библиотеки `torch`, реализуйте функцию активации ELU:

![](https://wikimedia.org/api/rest_v1/media/math/render/svg/eb23becd37c3602c4838e53f532163279192e4fd)

Создайте матрицу размера 4x3, заполненную числами из стандартного нормального распределения, и проверьте работоспособность функции активации.

In [ ]:
t = th.normal(0, 1, size=(4, 3))
t

tensor([[-0.6436, -2.8955,  0.8061],
        [ 1.8486,  2.5965, -1.1859],
        [-2.0361,  1.0684, -1.3147],
        [ 1.0728, -0.1180, -0.8238]])

In [ ]:
class ELU:
    def __init__(self, alpha: float):
        self.alpha = alpha

    def forward(self, inputs):
        return th.where(inputs < 0, self.alpha*(th.exp(inputs) - 1), inputs)

In [ ]:
ELU(alpha=1).forward(t)

tensor([[-0.4746, -0.9447,  0.8061],
        [ 1.8486,  2.5965, -0.6945],
        [-0.8695,  1.0684, -0.7314],
        [ 1.0728, -0.1113, -0.5612]])

### Создание функции потерь

<p class="task" id="3_1"></p>

3\.1 Используя операции над матрицами и векторами из библиотеки `torch`, реализуйте функцию потерь CrossEntropyLoss:

$$y_i = (y_{i,1},...,y_{i,k})$$
<img src="https://i.ibb.co/93gy1dN/Screenshot-9.png" width="200">

$$ CrossEntropyLoss = \frac{1}{n}\sum_{i=1}^{n}{L_i}$$
где $y_i$ - вектор правильных ответов для примера $i$, $\hat{y_i}$ - вектор предсказаний модели для примера $i$; $k$ - количество классов, $n$ - количество примеров в батче.

Создайте полносвязный слой с 3 нейронами и прогнать через него батч `inputs`. Полученный результат пропустите через функцию активации Softmax. Посчитайте значение функции потерь, трактуя вектор `y` как вектор правильных ответов.

In [ ]:
class CrossEntropyLoss:
  def forward(self, y_pred, y_true):
    return 1/y_pred.shape[0] * (-(y_true * th.log(y_pred)).sum())

In [ ]:
inputs = th.tensor([[1, 2, 3, 2.5],
                    [2, 5, -1, 2],
                    [-1.5, 2.7, 3.3, -0.8]])
y = th.tensor([1, 0, 0])

In [ ]:
y_pred = Linear(n_neurons=3, n_features=inputs.shape[1]).forward(inputs)
y_pred = Softmax().forward(y_pred)
y_pred

tensor([[9.9188e-01, 7.7648e-03, 3.5514e-04],
        [2.2327e-02, 1.7871e-02, 9.5980e-01],
        [7.3166e-01, 7.5316e-04, 2.6759e-01]])

In [ ]:
CrossEntropyLoss().forward(y_pred, y)

tensor(1.3742)

<p class="task" id="3_2"></p>

3.2 Модифицируйте MSE, добавив L2-регуляризацию.

$$MSE_R = MSE + \lambda\sum_{i=1}^{m}w_i^2$$

где $\lambda$ - коэффициент регуляризации; $w_i$ - веса модели.

In [ ]:
class MSERegularized:
    def __init__(self, lambda_):
        self.lambda_ = lambda_

    def data_loss(self, y_pred, y_true):
        return ((y_pred - y_true)**2).mean()

    def reg_loss(self, weights):
        return self.lambda_ * (weights**2).sum()

    def forward(self, y_pred, y_true, weights):
        return self.data_loss(y_pred, y_true) + self.reg_loss(weights)

In [ ]:
y_pred = th.tensor([-0.5, 1, 1.7])
y_true = th.tensor([0, 0.6, 2.3])
weights = th.normal(0, 5, (10, 1))

MSERegularized(lambda_=1).forward(y_pred, y_true, weights)

tensor(171.4246)

## Обратная связь
- [ ] Хочу получить обратную связь по решению